In [3]:
import os
import yaml
import csv
from collections import defaultdict

# Path to your top-level data folder
root_Dir = r"D:\DA\Stock Analysis\Stock_Data"
all_records=[]

# Store entries for each symbol(dictionary to collect ticker-wise data)
symbol_data = defaultdict(list)

# Traverse month folders
for month_folder in os.listdir(root_Dir):
    month_path = os.path.join(root_Dir, month_folder)
    if not os.path.isdir(month_path):
        continue

    # Traverse YAML files inside each month
    for filename in os.listdir(month_path):
        if filename.endswith(".yaml"):
            file_path = os.path.join(month_path, filename)
            with open(file_path, "r") as file:
                try:
                    entries = yaml.safe_load(file)
                    if not isinstance(entries, list):
                        print(f"Different format in {file_path}, So, skipping it")
                        continue
                except yaml.YAMLError as e:
                    print(f"Failed to parse {file_path}: {e}")
                    continue

                for entry in entries:
                    ticker = entry.get("Ticker")
                    if ticker:
                        symbol_data[ticker].append(entry)

# Output directory for CSVs
OUTPUT_DIR = "Stock_Data_CSV"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Write data per symbol
for ticker, records in symbol_data.items():
    output_file = os.path.join(OUTPUT_DIR, f"{ticker}.csv")
    
    if not records:
        continue

    # Use all possible keys as headers
    headers = sorted(set().union(*(record.keys() for record in records)))

    with open(output_file, "w", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for record in records:
            writer.writerow(record)
    # Add ticker info and store for combined CSV
    for record in records:
        record["Ticker"] = ticker  # Ensure Ticker is included
        all_records.append(record)

# Write combined CSV
if all_records:
    combined_headers = sorted(set().union(*(r.keys() for r in all_records)))
    combined_csv_path = os.path.join(r"D:\DA\Stock Analysis\Stock_Data", "all_tickers_combined.csv")
    with open(combined_csv_path, "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=combined_headers)
        writer.writeheader()
        for record in all_records:
            writer.writerow(record)

print("CSV files created successfully based on the symbols")


CSV files created successfully based on the symbols


SQL Data Storage

In [ ]:
%pip install mysql-connector-python

In [5]:
import mysql.connector
import pandas as pd

# Establish connection to the MySQL database in XAMPP
connection = mysql.connector.connect(
    host='localhost',       # XAMPP MySQL localhost
    user='root',            # Default user for MySQL in XAMPP
    password=''            # Default password for MySQL in XAMPP (usually empty)
)

cursor = connection.cursor()

# Create a new database
cursor.execute("CREATE DATABASE IF NOT EXISTS StockData")

# Commit changes (though in this case, commit is optional)
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Database created successfully!")

Database created successfully!


In [14]:
import mysql.connector
import pandas as pd

# Establish connection to the MySQL database in XAMPP
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password=''
)
# Create a cursor object to interact with the MySQL server
cursor = connection.cursor()

# Check if the database exists and create it if not
cursor.execute("CREATE DATABASE IF NOT EXISTS StockData")

# Switch to the newly created database
cursor.execute("USE StockData")

# Create a table named 'stock'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stock (
        ticker VARCHAR(100),
        close FLOAT NOT NULL,
        date VARCHAR(100),
        high FLOAT NOT NULL,
        low FLOAT NOT NULL,
        month VARCHAR(100),
        open FLOAT NOT NULL,
        volume INT
    )
''')

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Table created successfully!")

Table created successfully!


In [16]:
import mysql.connector
import csv

# Establish a connection to MySQL database
connection = mysql.connector.connect(
    host="localhost",  
    user="root",  
    password="",  
    database="StockData" 
)

# Create a cursor object to interact with the database
cursor = connection.cursor()

# Open and read the CSV file
with open(r'D:\DA\Stock Analysis\Stock_Data\all_tickers_combined.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row 

    # Iterate over the rows in the CSV file and insert them into the table
    for row in csv_reader:
        # Insert data into the movies table
        cursor.execute('''
            INSERT INTO stock (ticker, close, date, high, low, month, open, volume)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ''', (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]))  

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Data from CSV has been inserted into the users table successfully!")

Data from CSV has been inserted into the users table successfully!


In [18]:
#Sector data storage
import mysql.connector
import pandas as pd

# Establish connection to the MySQL database in XAMPP
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database="StockData"
)
# Create a cursor object to interact with the MySQL server
cursor = connection.cursor()

# Switch to the newly created database
cursor.execute("USE StockData")

# Create a table named 'sectpr'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sector (
        company VARCHAR(100),
        sector VARCHAR(100),
        symbol VARCHAR(100)
    )
''')

# Open and read the CSV file
with open(r'D:\DA\Stock Analysis\Stock_Data\Sector_data.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row 

    # Iterate over the rows in the CSV file and insert them into the table
    for row in csv_reader:
        # Insert data into the movies table
        cursor.execute('''
            INSERT INTO sector (company, sector, symbol)
            VALUES (%s, %s, %s)
        ''', (row[0], row[1], row[2]))  

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()

print("Data from CSV has been inserted into the users table successfully!")

Data from CSV has been inserted into the users table successfully!
